In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
from graphframes import *
from google.cloud import storage
import pickle
import pyspark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'project_ir313254724_322769175_pageview'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [ ]:
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path)
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# # Download the file (2.3GB)
!wget -N $pv_path
# # Filter for English pages, and keep just two fields: article ID (3) and monthly
# # total number of page views (5). Then, remove lines with article id or page
# # view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# # Create a Counter (dictionary) that sums up the pages views for the same
# # article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
    for line in f:
        parts = line.split(' ')
        wid2pv.update({int(parts[0]): int(parts[1])})

In [ ]:
name = "pageview"
with open(f"{name}.pkl", "wb") as f:
    pickle.dump(pr_dict, f)

client = storage.Client()
bucket = client.bucket(bucket_name)
blob_posting_locs = bucket.blob(f"page_view/{name}.pkl")
blob_posting_locs.upload_from_filename(f"{name}.pkl")